# J. Paul Getty Museum Collection Download
The museum is sharing an art collection dataset with some CC0 license, what we can use to train our CC0 ResNet backbone for other projects!
Their docs page: https://data.getty.edu/museum/collection/docs/ contains documentation how to download the images from their API.
We can use the image as long as it has a valid '"id": "https://creativecommons.org/publicdomain/zero/1.0/"' field in its data, what we will filter on. The dataset may contain up to 80000 images, and we will need a lot of that to train a ResNet backbone that would have transferable knowledge.

Note that their API is just awful, the server randomly time-outs but we do need the ids!

### Compatibility layer for SPARQLClient
sparql-client is outdated, made to work with python 3.1. With some trickery, we're making it work with 3.10+.

In [6]:
import base64
import json

def encodestring_f(x): 
    return x

def decodestring_f(x): 
    return x

base64.encodestring = encodestring_f
base64.decodestring = decodestring_f

import collections 
collections.MutableMapping = collections.abc.MutableMapping

import sparql

def _request_sparql(self, statement, timeout=0):
    query = self._queryString(statement)
    buf = sparql.tempfile.NamedTemporaryFile()

    opener = sparql.ev_request.build_opener(sparql.RedirectHandler)
    opener.addheaders = list(self.headers().items())
    try:
        response = self._build_response(query, opener, buf, timeout)
    except sparql.SparqlException as error:
        self.endpoint = error.message
        response = self._build_response(query, opener, buf, timeout)

    self._read_response(response, buf, timeout)
    buf.seek(0)
    return buf

def _build_request_sparql(self, query):
    if self.method == "GET":
        if '?' in self.endpoint:
            separator = '&'
        else:
            separator = '?'
        uri = self.endpoint.strip() + separator + query
        return sparql.ev_request.Request(uri)
    else:
        # uri = self.endpoint.strip().encode('ASCII')
        uri = self.endpoint.strip()
        data = query.encode()
        return sparql.ev_request.Request(uri, data=data)


sparql._Query._request = _request_sparql
sparql._Query._build_request = _build_request_sparql

### Constants and imports

In [2]:
import requests
import os
import random
from tqdm.notebook import tqdm
import pandas as pd
from atomicwrites import atomic_write

SPARQL_ENDPOINT = 'https://data.getty.edu/museum/collection/sparql'
METADATA_DIRECTORY = 'metadata'
DATASET_NAME = 'get'
RAW_METADATA_DIRECTORY = os.path.join(METADATA_DIRECTORY, 'raw')
RAW_METADATA_DIRECTORY_FILENAME = os.path.join(RAW_METADATA_DIRECTORY, f'{DATASET_NAME}.csv')
RAW_IMAGE_DIRECTORY = os.path.join('dataset', 'raw')
DATASET_IMAGES_LIMIT = 70000
DOWNLOADING_SAVE_PERIOD = 5

SPARQL_QUERY = 'PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> \n PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> \n SELECT DISTINCT ?sub  WHERE { \n ?sub ?pred ?obj . \n FILTER (regex(str(?sub), "collection/object") && ?ctt = 6). \n bind(((strlen(str(?sub)) - strlen(replace(str(?sub), "/", ""))) / strlen("/")) as ?ctt) \n } \n ORDER BY ?sub'


### Downloading metadata

In [3]:
def prepare_sparql(limit, offset):
    return f'{SPARQL_QUERY} LIMIT {limit} OFFSET {offset}'

In [4]:
s = sparql.Service(SPARQL_ENDPOINT, "utf-8", "POST")

In [5]:
query = s.query(prepare_sparql(1, 1))

RemoteDisconnected: Remote end closed connection without response